# Exercise 2

Try replacing GridSearchCV with RandomizedSearchCV.

In [19]:
import pandas as pd

dataset = pd.read_csv('Housing.csv')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [20]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset, test_size = 0.2, random_state = 42)

In [21]:
# Create income categories. We want to make sure the test set is representative of the incomes in the whole dataset.

from sklearn.model_selection import StratifiedShuffleSplit

dataset["income_cat"] = np.ceil(dataset["median_income"] / 1.5)
dataset["income_cat"].where(dataset["income_cat"] < 5, 5.0, inplace=True)

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(dataset, dataset["income_cat"]):
    strat_train_set = dataset.loc[train_index]
    strat_test_set = dataset.loc[test_index]
    
dataset["income_cat"].value_counts() / len(dataset)

# Drop income_cat so it's back to its original state
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)
    
dataset = strat_train_set.copy()

In [22]:
# Revert to a clean training set to prepare for ML models

dataset = strat_train_set.drop("median_house_value", axis = 1)
housing_labels = strat_train_set["median_house_value"].copy()

from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = "median")
housing_num = dataset.drop("ocean_proximity", axis = 1)
imputer.fit(housing_num)
print(imputer.statistics_) # These are the medians of each attribute

[-118.51     34.26     29.     2119.5     433.     1164.      408.
    3.5409]


In [23]:
# Definition of the CategoricalEncoder class, copied from PR #9151.
# Just run this cell, or copy it to your code, do not try to understand it (yet).

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y = None):
        return self
    def transform(self, X, y = None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household]
        else: 
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room = False)
housing_extra_attribs = attr_adder.transform(dataset.values)

In [25]:
# Pipeline constructor takes a list of name/estimator pairs defining a sequence of steps
# When you call a pipeline's fit() method, it calls fit_transform() sequentially on all transformers
# For data that needs special treatment, we can write a special pipeline for it

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy = 'median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('cat_encoder', CategoricalEncoder(encoding="onehot-dense")),
])

housing_num_tr = num_pipeline.fit_transform(housing_num)
print(housing_num_tr)

[[-1.15604281  0.77194962  0.74333089 ... -0.61493744 -0.31205452
  -0.08649871]
 [-1.17602483  0.6596948  -1.1653172  ...  1.33645936  0.21768338
  -0.03353391]
 [ 1.18684903 -1.34218285  0.18664186 ... -0.5320456  -0.46531516
  -0.09240499]
 ...
 [ 1.58648943 -0.72478134 -1.56295222 ... -0.3167053   0.3469342
  -0.03055414]
 [ 0.78221312 -0.85106801  0.18664186 ...  0.09812139  0.02499488
   0.06150916]
 [-1.43579109  0.99645926  1.85670895 ... -0.15779865 -0.22852947
  -0.09586294]]


In [26]:
# Join the pipelines using a FeatureUnion

from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list = [
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline)
])

housing_prepared = full_pipeline.fit_transform(dataset)
housing_prepared.shape

(16512, 15)

In [27]:
# Use a random search here instead of a grid search

from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal

param_distribs = {
    'kernel': ['linear', 'rbf'], 
    'C': reciprocal(20, 200000)
}
svr_reg = SVR()
randomized_search = RandomizedSearchCV(
    estimator = svr_reg,
    param_distributions = param_distribs, 
    cv = 5,
    n_iter = 50,
    n_jobs=4,
    verbose=2,
    scoring = 'neg_mean_squared_error'
)

randomized_search.fit(housing_prepared, housing_labels)
print(randomized_search.best_params_)
print(randomized_search.best_estimator_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] C=282.0145589872492, kernel=rbf .................................
[CV] C=282.0145589872492, kernel=rbf .................................
[CV] C=282.0145589872492, kernel=rbf .................................
[CV] C=282.0145589872492, kernel=rbf .................................
[CV] .................. C=282.0145589872492, kernel=rbf, total=   8.5s
[CV] C=282.0145589872492, kernel=rbf .................................
[CV] .................. C=282.0145589872492, kernel=rbf, total=   8.4s
[CV] C=4732.913630966699, kernel=rbf .................................
[CV] .................. C=282.0145589872492, kernel=rbf, total=   8.6s
[CV] C=4732.913630966699, kernel=rbf .................................
[CV] .................. C=282.0145589872492, kernel=rbf, total=   8.5s
[CV] C=4732.913630966699, kernel=rbf .................................
[CV] .................. C=282.0145589872492, kernel=rbf, total=   8.4s
[CV] ..........

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  2.1min


[CV] ................. C=10241.815445279695, kernel=rbf, total=   7.3s
[CV] C=4101.437115574346, kernel=rbf .................................
[CV] ................. C=10241.815445279695, kernel=rbf, total=   7.7s
[CV] C=4101.437115574346, kernel=rbf .................................
[CV] .................. C=4101.437115574346, kernel=rbf, total=   7.8s
[CV] C=4101.437115574346, kernel=rbf .................................
[CV] .................. C=4101.437115574346, kernel=rbf, total=   7.9s
[CV] C=30756.604350802936, kernel=rbf ................................
[CV] .................. C=4101.437115574346, kernel=rbf, total=   7.8s
[CV] C=30756.604350802936, kernel=rbf ................................
[CV] .................. C=4101.437115574346, kernel=rbf, total=   7.8s
[CV] C=30756.604350802936, kernel=rbf ................................
[CV] .................. C=4101.437115574346, kernel=rbf, total=   8.5s
[CV] C=30756.604350802936, kernel=rbf ................................
[CV] .

[CV] .................. C=64.12624900894008, kernel=rbf, total=   7.9s
[CV] C=116.02056786055311, kernel=rbf ................................
[CV] .................. C=64.12624900894008, kernel=rbf, total=   8.0s
[CV] C=116.02056786055311, kernel=rbf ................................
[CV] .................. C=64.12624900894008, kernel=rbf, total=   7.8s
[CV] C=116.02056786055311, kernel=rbf ................................
[CV] .................. C=64.12624900894008, kernel=rbf, total=   7.9s
[CV] C=116.02056786055311, kernel=rbf ................................
[CV] ................. C=116.02056786055311, kernel=rbf, total=   7.7s
[CV] C=116.02056786055311, kernel=rbf ................................
[CV] ................. C=116.02056786055311, kernel=rbf, total=   7.8s
[CV] C=303.41635140361103, kernel=rbf ................................
[CV] ................. C=116.02056786055311, kernel=rbf, total=   7.7s
[CV] C=303.41635140361103, kernel=rbf ................................
[CV] .

[CV] .............. C=160.72778084221562, kernel=linear, total=   5.3s
[CV] C=194.6174925023465, kernel=rbf .................................
[CV] .................. C=194.6174925023465, kernel=rbf, total=   7.8s
[CV] C=194.6174925023465, kernel=rbf .................................
[CV] .................. C=194.6174925023465, kernel=rbf, total=   7.7s
[CV] C=31129.431012054523, kernel=linear .............................
[CV] .................. C=194.6174925023465, kernel=rbf, total=   7.6s
[CV] C=31129.431012054523, kernel=linear .............................
[CV] .................. C=194.6174925023465, kernel=rbf, total=   7.6s
[CV] C=31129.431012054523, kernel=linear .............................


[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  8.3min


[CV] .................. C=194.6174925023465, kernel=rbf, total=   7.6s
[CV] C=31129.431012054523, kernel=linear .............................
[CV] .............. C=31129.431012054523, kernel=linear, total=  11.0s
[CV] C=31129.431012054523, kernel=linear .............................
[CV] .............. C=31129.431012054523, kernel=linear, total=   9.9s
[CV] C=36594.60238414861, kernel=rbf .................................
[CV] .............. C=31129.431012054523, kernel=linear, total=  12.2s
[CV] C=36594.60238414861, kernel=rbf .................................
[CV] .............. C=31129.431012054523, kernel=linear, total=  11.6s
[CV] C=36594.60238414861, kernel=rbf .................................
[CV] .............. C=31129.431012054523, kernel=linear, total=  10.6s
[CV] C=36594.60238414861, kernel=rbf .................................
[CV] .................. C=36594.60238414861, kernel=rbf, total=   7.4s
[CV] C=36594.60238414861, kernel=rbf .................................
[CV] .

[CV] ............... C=47867.84801514598, kernel=linear, total=  15.3s
[CV] C=49628.35162906833, kernel=rbf .................................
[CV] ............... C=47867.84801514598, kernel=linear, total=  14.3s
[CV] C=49628.35162906833, kernel=rbf .................................
[CV] .................. C=49628.35162906833, kernel=rbf, total=   7.6s
[CV] C=49628.35162906833, kernel=rbf .................................
[CV] ............... C=47867.84801514598, kernel=linear, total=  13.8s
[CV] C=49628.35162906833, kernel=rbf .................................
[CV] .................. C=49628.35162906833, kernel=rbf, total=   7.5s
[CV] C=1824.3825951583308, kernel=linear .............................
[CV] .................. C=49628.35162906833, kernel=rbf, total=   7.6s
[CV] C=1824.3825951583308, kernel=linear .............................
[CV] .............. C=1824.3825951583308, kernel=linear, total=   6.0s
[CV] C=1824.3825951583308, kernel=linear .............................
[CV] .

[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed: 12.7min finished


{'C': 190329.41415595863, 'kernel': 'rbf'}
SVR(C=190329.41415595863, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto', kernel='rbf', max_iter=-1, shrinking=True, tol=0.001,
  verbose=False)


In [29]:
# Evaluate the final model on the test set
from sklearn.metrics import mean_squared_error

final_model = randomized_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis = 1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print(final_rmse)

55382.19144525687


In [ ]:
# Good, but not quite as good as the Random Forest